In [ ]:
import os,keras,cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
import skimage.io as io
import math
from os import remove
from PIL import Image 
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import NASNetMobile
from keras.models import load_model
import time

In [ ]:
def get_iou(bb1, bb2): ## calcular iou
    assert bb1['x1'] < bb1['x2'] ## lanzan errores si esta mal.
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top) ## me voy a quedar solo con el area de interseccion
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area /  bb2_area ## bb2_area es siempre la peuqeña
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [ ]:
path = "/Data/Train/"
all_images = os.listdir(path)
print("Numero total de imagenes")
len(all_images)

In [ ]:
df = pd.read_csv("InfoBacilos-separado.csv")

In [ ]:
train_images=[]
train_labels=[]
name_labels=[]
aumento=20
for i,filename in enumerate(all_images):
        NumBusqueda = 0
        image = cv2.imread(f'./Data/Train/{filename}')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if df[df['Imagenes'] == filename].Imagenes.any():
            NAnotaciones=df[df['Imagenes'] == filename]
            Minimo=image.min()
            Maximo=image.max()
            image= (((image - Minimo)/(Maximo-Minimo))*255)
            image = image.astype(np.uint8)
            gtvalues=[]
            imout = image.copy()
            for index, row in NAnotaciones.iterrows():
               # print(row['Xmin'])
                x1=row['Xmin']
                x2=row['Xmax']
                y1=row['Ymin']
                y2=row['Ymax']
                gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
                area= (x2-x1)*(y2-y1)
    

            X1 = 23
            X2 = 63
            Y1 = 23
            Y2 = 63 
            flag = 0
            while flag == 0:
                if X2>1584:
                    X1 = 23
                    X2 = 63
                    Y1=Y1+40      
                    Y2=Y2+40
                    if Y2>1203:
                        flag=1
   
                iou_max=0
                for gtval in gtvalues:
                    iou = get_iou(gtval,{"x1":X1,"x2":X2,"y1":Y1,"y2":Y2})
                    if iou >iou_max:
                         iou_max= iou


                if iou_max > 0.5:
                       
                        #timage = imout[(Y1):(Y2),(X1):(X2)]
                        timage = imout[(Y1-aumento):(Y2+aumento),(X1-aumento):(X2+aumento)] # y comprobar que sea mas grande..
                        # las coerdenadas son estaspero voy a aoumertar

                        resized = cv2.resize(timage, (80,80), interpolation = cv2.INTER_NEAREST)
                        train_images.append(resized)
                        train_labels.append(1)
                        name_labels.append(filename)
                
                else :
                        timage = imout[(Y1-aumento):(Y2+aumento),(X1-aumento):(X2+aumento)]
                        resized = cv2.resize(timage, (80,80), interpolation = cv2.INTER_NEAREST)
                        train_images.append(resized)
                        train_labels.append(0)
                       
                X1=X1+40      
                X2=X2+40
              



In [ ]:
X_new = np.array(train_images)
y_new = np.array(train_labels)
X_new=X_new/255


In [ ]:
del train_images, train_labels
sum(y_new)

In [ ]:

path = "/Data/Validation/"
all_images = os.listdir(path)
print("Numero total de imagenes")
len(all_images) 
validation_images=[]
validation_labels=[]

aumento=20
for i,filename in enumerate(all_images):
        NumBusqueda = 0
        image = cv2.imread(f'./Data/Validation/{filename}')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if df[df['Imagenes'] == filename].Imagenes.any():
            NAnotaciones=df[df['Imagenes'] == filename]
            Minimo=image.min()
            Maximo=image.max()
            image= (((image - Minimo)/(Maximo-Minimo))*255)
            image = image.astype(np.uint8)

            gtvalues=[]
            imout = image.copy()
            for index, row in NAnotaciones.iterrows():
                x1=row['Xmin']
                x2=row['Xmax']
                y1=row['Ymin']
                y2=row['Ymax']
                gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
                area= (x2-x1)*(y2-y1)        
            X1 = 23
            X2 = 63
            Y1 = 23
            Y2 = 63 
            flag = 0
            while flag == 0:
                if X2>1584:
                    X1 = 23
                    X2 = 63
                    Y1=Y1+40      
                    Y2=Y2+40
                    if Y2>1203:
                        flag=1

                
                iou_max=0
                for gtval in gtvalues:
                    iou = get_iou(gtval,{"x1":X1,"x2":X2,"y1":Y1,"y2":Y2})
                    if iou >iou_max:
                         iou_max= iou


                if iou_max > 0.5:
                       
                        timage = imout[(Y1-aumento):(Y2+aumento),(X1-aumento):(X2+aumento)] # y comprobar que sea mas grande..
           
                        resized = cv2.resize(timage, (80,80), interpolation = cv2.INTER_NEAREST)
                        validation_images.append(resized)
                        validation_labels.append(1)


                else :
                        timage = imout[(Y1-aumento):(Y2+aumento),(X1-aumento):(X2+aumento)]
                        resized = cv2.resize(timage, (80,80), interpolation = cv2.INTER_NEAREST)
                        validation_images.append(resized)
                        validation_labels.append(0)
       
                X1=X1+40      
                X2=X2+40
              
X_val = np.array(validation_images)
y_val = np.array(validation_labels)
X_val=X_val/255


In [ ]:
del validation_images, validation_labels
sum(y_val)

In [ ]:
basemodel = NASNetMobile(
    include_top = False, 
    weights = 'imagenet', 
    input_tensor = Input((80, 80, 3))#VGG entradas 224?
)
basemodel.trainable = True

In [ ]:

imputs = tf.keras.Input(shape=(80,80,3))
prcesado= Conv2D (8,18,padding="same",activation='relu', input_shape=(80,80,3))(imputs)
prcesado= MaxPooling2D (pool_size=(2, 2))(prcesado)
prcesado = Dropout(0.1)(prcesado)
prcesado= Conv2D (32,18,padding="same",activation='relu')(prcesado)
prcesado= MaxPooling2D (pool_size=(2, 2))(prcesado)
prcesado = Dropout(0.1)(prcesado)
prcesado= Conv2D (64,18,padding="same",activation='relu')(prcesado)
headmodel = basemodel.layers[-2].output
headmodel = GlobalAveragePooling2D()(headmodel)
headmodel = Flatten()(headmodel) 
headmodel = Dense(128, activation = "elu")(headmodel)
headmodel = Dropout(0.1)(headmodel)
predictions = Dense(2, activation="softmax")(headmodel)
model_final = Model( basemodel.inputs, predictions)

MyList = ["accuracy"]
MyList += [Recall(class_id = i) for i in range(2)] 
MyList += [Precision(class_id = i) for i in range(2)]
model_final.compile(optimizer = Adam(learning_rate=0.0001),loss = keras.losses.poisson, metrics=MyList)  ## se puede hacer como categorical cross entropy
#model_final.summary()

In [ ]:
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)
lenc = MyLabelBinarizer()
Y =  lenc.fit_transform(y_new)
Y_val =  lenc.fit_transform(y_val)

In [ ]:
checkpoint = ModelCheckpoint("rcnn_MNasNet_2.h5", monitor='loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto')#, period=1)
inicio =time.time()
print("inicio:", inicio)
hist = model_final.fit(x= X_new,y = Y, batch_size= 32, validation_data =(X_val, Y_val),epochs= 50, callbacks=[checkpoint])